In [43]:
import json
import numpy as np
from itertools import product
from branching_sus.implementation import ConvexGraphBranching, SubsetSimulation
from branching_sus.performance_function import breitung, himmel
from branching_sus.estimate import exceedance_probability
from scipy.spatial.distance import euclidean

In [30]:
def run_bss_experiment(performance_function,
                       threshold,
                       region_indicator,
                       file_name,
                       dimensions,
                       sizes,
                       graph_budgets,
                       seeds,
                       penalties,
                       regs):
    results_keys = [
        'dimension',
        'size',
        'graph_budget',
        'seed',
        'penalty',
        'reg',
        'region_indicator',
        'performance_evaluation',
        'exceedance_estimate'
    ]
    results = {key:[] for key in results_keys}
    param_keys = [
        'dimension',
        'size',
        'graph_budget',
        'seed',
        'penalty',
        'reg'
    ]
    param_iterator = product(*[
        dimensions,
        sizes,
        graph_budgets,
        seeds,
        penalties,
        regs,
    ])
    for param_set in param_iterator:
        
        param_dict = {key:value
                      for key, value in zip(param_keys,param_set)}
        
        for key, value in param_dict.items():
            results[key].append(value)
        
        lsvc_params = {
            'penalty': param_dict['penalty'],
            'C': param_dict['reg'],
            'dual':'auto'
        }
        
        bss = ConvexGraphBranching(performance_function=performance_function,
                                   dimension=param_dict['dimension'],
                                   level_size=param_dict['size'],
                                   threshold=threshold,
                                   level_probability=0.1,
                                   seed=param_dict['seed'],
                                   params=lsvc_params,
                                   convex_budget=param_dict['graph_budget'],
                                   verbose=False)
  
        bss.run()
        
        results['region_indicator'].append(region_indicator(bss))
        results['performance_evaluation'].append(bss
                                                 .initial_level
                                                 .indicator
                                                 .performance_function
                                                 .eval_count)
        results['exceedance_estimate'].append(exceedance_probability(bss,threshold))
        
        with open(file_name, 'w') as file:
            json.dump(results, file)
                

In [31]:
def run_sus_experiment(performance_function,
                       threshold,
                       region_indicator,
                       file_name,
                       dimensions,
                       sizes,
                       seeds):
    results_keys = [
        'dimension',
        'size',
        'seed',
        'region_indicator',
        'performance_evaluation',
        'exceedance_estimate'
    ]
    results = {key:[] for key in results_keys}
    param_keys = [
        'dimension',
        'size',
        'seed',
    ]
    param_iterator = product(*[
        dimensions,
        sizes,
        seeds,
    ])
    for param_set in param_iterator:
        
        param_dict = {key:value
                      for key, value in zip(param_keys,param_set)}
        
        for key, value in param_dict.items():
            results[key].append(value)
        
        sus = SubsetSimulation(performance_function=performance_function,
                               dimension=param_dict['dimension'],
                               level_size=param_dict['size'],
                               threshold=threshold,
                               level_probability=0.1,
                               seed=param_dict['seed'],
                               verbose=False)
  
        sus.run()
        
        results['region_indicator'].append(region_indicator(sus))
        results['performance_evaluation'].append(sus
                                                 .initial_level
                                                 .indicator
                                                 .performance_function
                                                 .eval_count)
        results['exceedance_estimate'].append(exceedance_probability(sus,threshold))
        
        with open(file_name, 'w') as file:
            json.dump(results, file)

In [32]:
def breitung_region_indicator(bss):
    return any([euclidean(x.array[:2],(4,0)) < 0.25
                for x in bss.all_samples])
    
def himmel_region_indicator(bss):
    maxima = [
        (3, 2),
        (-2.805118, 3.131312),
        (-3.779310, -3.283186),
        (3.584428, -1.848126)
    ]
    def sample_indicator(x):
        return [euclidean(x[:2],maxi) < 0.25 for maxi in maxima]
    
    return [any(maxima)
            for maxima in list(zip(*[sample_indicator(x.array)
                                     for x in bss.all_samples]))]
    

# Breitung

In [ ]:
dimensions = [2]
sizes = [100,250,500,750,1000,1250,1500,2000,3000]
seeds = [i for i in range(100)]

run_sus_experiment(performance_function=breitung,
                   threshold=0,
                   region_indicator=breitung_region_indicator,
                   file_name="bre_sus_dim_2.json",
                   dimensions=dimensions,
                   sizes=sizes,
                   seeds=seeds)

In [ ]:
dimensions = [2]
sizes = [500,1000,1500]
graph_budgets = [100,500,1000]
seeds = [i for i in range(100)]
penalties = ['l2']
regs = [1]

run_bss_experiment(performance_function=breitung,
                   threshold=0,
                   region_indicator=breitung_region_indicator,
                   file_name="bre_bss_dim_2.json",
                   dimensions=dimensions,
                   sizes=sizes,
                   graph_budgets=graph_budgets,
                   seeds=seeds,
                   penalties=penalties,
                   regs=regs)

In [ ]:
dimensions = [50]
sizes = [500,1000,1500]
graph_budgets = [100,500,1000]
seeds = [i for i in range(100)]
penalties = ['l2']
regs = [1]

run_bss_experiment(performance_function=breitung,
                   threshold=0,
                   region_indicator=breitung_region_indicator,
                   file_name="bre_bss_dim_50.json",
                   dimensions=dimensions,
                   sizes=sizes,
                   graph_budgets=graph_budgets,
                   seeds=seeds,
                   penalties=penalties,
                   regs=regs)

In [ ]:
dimensions = [50]
sizes = [500]
graph_budgets = [1000]
seeds = [i for i in range(100)]
penalties = ['l1']
regs = [0.5,2,5,10]

run_bss_experiment(performance_function=breitung,
                   threshold=0,
                   region_indicator=breitung_region_indicator,
                   file_name="bre_bss_dim_50_l1.json",
                   dimensions=dimensions,
                   sizes=sizes,
                   graph_budgets=graph_budgets,
                   seeds=seeds,
                   penalties=penalties,
                   regs=regs)

In [ ]:
dimensions = [50]
sizes = [500]
graph_budgets = [1000]
seeds = [i for i in range(100)]
penalties = ['l2']
regs = [0.5,2,5,10]


run_bss_experiment(performance_function=breitung,
                   threshold=0,
                   region_indicator=breitung_region_indicator,
                   file_name="bre_bss_dim_50_l2.json",
                   dimensions=dimensions,
                   sizes=sizes,
                   graph_budgets=graph_budgets,
                   seeds=seeds,
                   penalties=penalties,
                   regs=regs)

# Himmelblau

In [ ]:
dimensions = [2]
sizes = [100,500,1000,1500,2000]
seeds = [i for i in range(100)]

run_sus_experiment(performance_function=himmel,
                   threshold=np.inf,
                   region_indicator=himmel_region_indicator,
                   file_name="him_sus_dim_2.json",
                   dimensions=dimensions,
                   sizes=sizes,
                   seeds=seeds)

In [ ]:
dimensions = [2]
sizes = [100,500,1000]
graph_budgets = [100,250,500]
seeds = [i for i in range(100)]
penalties = ['l2']
regs = [1]

run_bss_experiment(performance_function=himmel,
                   threshold=np.inf,
                   region_indicator=himmel_region_indicator,
                   file_name="him_bss_dim_2.json",
                   dimensions=dimensions,
                   sizes=sizes,
                   graph_budgets=graph_budgets,
                   seeds=seeds,
                   penalties=penalties,
                   regs=regs)

In [ ]:
dimensions = [50]
sizes = [100,500,1000]
graph_budgets = [100,250,500]
seeds = [i for i in range(100)]
penalties = ['l2']
regs = [1]

run_bss_experiment(performance_function=himmel,
                   threshold=np.inf,
                   region_indicator=himmel_region_indicator,
                   file_name="him_bss_dim_50.json",
                   dimensions=dimensions,
                   sizes=sizes,
                   graph_budgets=graph_budgets,
                   seeds=seeds,
                   penalties=penalties,
                   regs=regs)

In [ ]:
dimensions = [50]
sizes = [500]
graph_budgets = [100]
seeds = [i for i in range(100)]
penalties = ['l1']
regs = [0.5,2,5,10]

run_bss_experiment(performance_function=himmel,
                   threshold=np.inf,
                   region_indicator=himmel_region_indicator,
                   file_name="him_bss_dim_50_l1.json",
                   dimensions=dimensions,
                   sizes=sizes,
                   graph_budgets=graph_budgets,
                   seeds=seeds,
                   penalties=penalties,
                   regs=regs)

In [ ]:
dimensions = [50]
sizes = [500]
graph_budgets = [100]
seeds = [i for i in range(100)]
penalties = ['l2']
regs = [0.5,2,5,10]


run_bss_experiment(performance_function=himmel,
                   threshold=np.inf,
                   region_indicator=himmel_region_indicator,
                   file_name="him_bss_dim_50_l2.json",
                   dimensions=dimensions,
                   sizes=sizes,
                   graph_budgets=graph_budgets,
                   seeds=seeds,
                   penalties=penalties,
                   regs=regs)